# DeepSEA Demo
This notebook demonstrates the use of convolutional neural networks for adressing genomic problems.
It also shows how Kipoi can be used to get trained models for problems instead of having to train
your own.

Our example method for this demonstration will be [DeepSEA](https://www.nature.com/articles/nmeth.3547), which predicts the effects of noncoding genomic variants on epigenetic features.

## Running Locally

If you're running this notebook locally, you'll need to install the dependencies first. If you use [Conda](https://docs.conda.io/en/latest/miniconda.html) for your package manager, you can use the command `conda env create -f environment.yml` to install everything you need. The conda environment can then be made into a jupyter kernel with the command `python -m ipykernel install --user --name workshop`

In [1]:
# Import the necessary libraries
import kipoi
import torch

## Download the model and example data from Kipoi

I didn't know the code below off the top of my head.
Each Kipoi model page has descriptions of how to use the model.

<img src="kipoi_screenshot.png">

In [2]:
# Download trained model from Kipoi
model = kipoi.get_model('DeepSEA/predict')

Using downloaded and verified file: /home/heil/.kipoi/models/DeepSEA/predict/downloaded/model_files/weights/89e640bf6bdbe1ff165f484d9796efc7


In [3]:
# Download example dataloader kwargs
dl_kwargs = model.default_dataloader.download_example('example')
# Get the dataloader and instantiate it
dl = model.default_dataloader(**dl_kwargs)
# get a batch iterator
it = dl.batch_iter(batch_size=10)
# predict for a batch
batch = next(it)

## Data Tour
Always start by looking at your data

In [4]:
print(batch)

{'inputs': array([[[[1., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 1., 1., 0.]],

        [[0., 1., 0., ..., 0., 0., 0.]],

        [[0., 0., 1., ..., 0., 0., 1.]]],


       [[[0., 0., 1., ..., 0., 1., 0.]],

        [[0., 0., 0., ..., 0., 0., 1.]],

        [[1., 1., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 1., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 1., 0.]],

        [[0., 0., 1., ..., 0., 0., 1.]],

        [[1., 0., 0., ..., 1., 0., 0.]],

        [[0., 1., 0., ..., 0., 0., 0.]]],


       ...,


       [[[1., 1., 1., ..., 1., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 1., 1.]]],


       [[[0., 1., 1., ..., 1., 1., 1.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[1., 0., 0., ..., 0., 0., 0.]]],


       [[[1., 1., 1., ..., 1., 1., 0.]],

        [[0., 0., 0., ..., 0., 0., 1.]],

        [[0., 0., 0., ..., 0., 0., 0.]],


In [5]:
print(batch['inputs'].shape)

(10, 4, 1, 1000)


In [6]:
# Convert one-hot encoding to DNA
example_sequence_one_hot = batch['inputs'][0,:,:,:].squeeze()
print(example_sequence_one_hot.shape)

(4, 1000)


In [7]:
# Convert one-hot into integer labels
example_sequence_indices = example_sequence_one_hot.argmax(axis=0)
print(example_sequence_indices)

[0 2 3 1 3 2 1 0 1 0 2 0 0 1 0 3 3 3 3 2 0 2 0 3 3 3 2 3 3 3 3 1 3 1 0 2 3
 0 2 3 3 1 1 3 3 0 1 0 0 3 3 1 1 1 0 3 0 3 3 2 3 0 2 2 3 0 0 3 2 0 1 0 2 3
 3 3 3 0 0 3 2 1 1 0 1 3 2 0 0 0 3 0 0 0 0 0 1 0 3 0 0 0 2 3 2 0 1 3 2 1 1
 3 0 0 2 0 2 3 1 0 1 0 3 2 2 3 3 3 0 3 2 2 1 0 2 0 0 1 3 0 2 2 0 1 3 1 0 0
 2 1 1 1 0 2 3 3 1 3 3 1 3 2 0 0 0 3 1 1 0 0 0 3 1 1 0 2 2 1 1 0 1 3 3 3 1
 0 1 1 1 0 3 2 1 1 1 3 1 2 3 2 2 3 2 1 1 1 0 2 1 1 1 3 2 3 2 1 1 3 1 0 3 0
 0 1 1 1 2 2 2 3 2 1 3 2 0 2 2 3 2 2 0 2 1 0 2 1 3 1 1 1 0 3 1 1 0 2 2 3 1
 1 1 0 0 2 0 3 0 2 2 1 3 1 1 1 0 1 3 2 1 3 1 1 0 2 0 1 2 3 1 1 1 1 0 3 2 2
 0 3 3 2 1 1 1 1 1 0 2 1 0 0 0 2 2 0 1 1 3 1 1 3 2 1 1 0 2 1 0 2 1 3 1 1 2
 2 2 0 0 2 2 0 2 1 3 1 3 2 1 0 2 0 2 0 2 1 1 1 3 3 3 2 2 0 0 2 1 1 0 2 0 2
 1 0 2 0 0 0 2 2 2 0 2 1 0 2 2 1 0 1 1 3 2 1 3 2 2 0 1 0 0 1 3 2 2 2 1 1 3
 1 1 0 3 1 1 0 0 2 3 1 0 2 0 3 3 1 1 3 3 1 1 0 2 2 0 1 0 0 0 2 1 3 2 1 3 1
 3 1 1 3 3 1 0 1 1 1 0 2 0 0 1 0 1 1 0 1 3 2 3 1 0 1 1 3 1 1 3 1 0 1 0 2 0
 2 2 0 2 0 0 0 1 0 3 1 3 

In [8]:
# Kipoi models usually use ACGT, but that's not necessarily the case so be careful 
decoder = {0: 'A',
           1: 'C',
           2: 'G',
           3: 'T'
          }

# Decode the DNA sequence
decoded = []
for index in example_sequence_indices:
    decoded.append(decoder[index])
    
print(''.join(decoded))

AGTCTGCACAGAACATTTTGAGATTTGTTTTCTCAGTAGTTCCTTACAATTCCCATATTGTAGGTAATGACAGTTTTAATGCCACTGAAATAAAAACATAAAGTGACTGCCTAAGAGTCACATGGTTTATGGCAGAACTAGGACTCAAGCCCAGTTCTTCTGAAATCCAAATCCAGGCCACTTTCACCCATGCCCTCGTGGTGCCCAGCCCTGTGCCTCATAACCCGGGTGCTGAGGTGGAGCAGCTCCCATCCAGGTCCCAAGATAGGCTCCCACTGCTCCAGACGTCCCCATGGATTGCCCCCAGCAAAGGACCTCCTGCCAGCAGCTCCGGGAAGGAGCTCTGCAGAGAGCCCTTTGGAAGCCAGAGCAGAAAGGGAGCAGGCACCTGCTGGACAACTGGGCCTCCATCCAAGTCAGATTCCTTCCAGGACAAAGCTGCTCTCCTTCACCCAGAACACCACTGTCACCTCCTCACAGAGGAGAAACATCTTTGTTCTTCCATCTCAAAAGAGCTGGCTTTGCTGATATGACAGGCCCCAAAGAGCAAGTCAGCCTCATCAGCAGTTTTTCCTCCTCCCTCCTCCGCATTCTTCCTGGTGCGTCATCTTCCAAGGTGACACATACATTGTGGCTTTGGCAGGACTCCTGCCTGTTGGGACTCAGGAAGTTCACTTTGTCCTCCTAAGTCTCTATGTTGACACGCCCTTGCCTGTAAACACAAGAATTGAGAGGGGATATGATGATTCCAGAGATAGGAAATTGATCTCTAACCAAATTTCACATCTTAAGAAGGCCTGTGACTCTGGGACCACGGGTACCATGTTGAGAAGGGTTCCACCCAGTGGTCATGAGCACAGACCTTGTTCTCAGACCTGATTCCTCCAGGCAGGTTATTTGACATTTATGAACCTCAGTGTTCTCTGAAATGGGGATCATCCCCTGACTTCTGAGGGCAGTTAAATGAGATCAAGCATGTAAAGCTCTTAGCACCAAGCCT

### What should the outputs look like?

In [9]:
# This file comes from the resources folder of the DeepSEA standalone package
# that can be found here: http://deepsea.princeton.edu/media/code/deepsea.v0.94c.tar.gz
with open ('predictor.names') as feature_file:
    feature_names = [name.strip() for name in feature_file]

print(len(feature_names))
print(feature_names[:5])

919
['8988T|DNase|None', 'AoSMC|DNase|None', 'Chorion|DNase|None', 'CLL|DNase|None', 'Fibrobl|DNase|None']


## Making predictions with the data

In [10]:
pred = model.pipeline.predict(dl_kwargs, batch_size=10)

1it [00:00,  5.29it/s]


In [11]:
pred

array([[0.08160985, 0.06867626, 0.10076762, ..., 0.09493407, 0.02133884,
        0.01201438],
       [0.06698142, 0.01062412, 0.02694611, ..., 0.15490845, 0.04822356,
        0.00770111],
       [0.04445538, 0.00539725, 0.01840791, ..., 0.14994638, 0.3529725 ,
        0.02272797],
       ...,
       [0.00081359, 0.00404314, 0.00176917, ..., 0.02167308, 0.11304276,
        0.02273136],
       [0.00079004, 0.00249485, 0.00252153, ..., 0.05105   , 0.04735951,
        0.01576868],
       [0.00076793, 0.00237438, 0.00250099, ..., 0.05096852, 0.04723714,
        0.01515259]], dtype=float32)

In [12]:
print(pred.shape)

(10, 919)


In [13]:
# Get the first prediction from the batch
single_pred = torch.Tensor(pred[0,:])
print(single_pred.shape)

torch.Size([919])


In [14]:
values, indices = torch.topk(single_pred, 5)

for value, index in zip(values, indices):
    feature = feature_names[index]
    print('Feature {} has a probability of {}'.format(feature, value))

Feature NHEK|H3K4me1|None has a probability of 0.6889464259147644
Feature NH-A|H3K4me1|None has a probability of 0.6628297567367554
Feature Osteoblasts|H3K4me1|None has a probability of 0.6460651159286499
Feature NH-A|H3K4me2|None has a probability of 0.541836142539978
Feature NHEK|H3K4me2|None has a probability of 0.49794310331344604


### Sequence 1 Info
H3K4 methylation indicates active transcription

NHEK = Normal epithelial keratinocytes  
NH-A = Astrocytes

In [15]:
# Get a single prediction from the batch
single_pred = torch.Tensor(pred[5,:])
print(single_pred.shape)

torch.Size([919])


In [16]:
values, indices = torch.topk(single_pred, 5)

for value, index in zip(values, indices):
    feature = feature_names[index]
    print('Feature {} has a probability of {}'.format(feature, value))

Feature U2OS|SETDB1|None has a probability of 0.36347267031669617
Feature Monocytes-CD14+RO01746|H3K9me3|None has a probability of 0.19080045819282532
Feature K562|KAP1|None has a probability of 0.14699679613113403
Feature H1-hESC|H3K9me3|None has a probability of 0.12245297431945801
Feature HEK293|KAP1|None has a probability of 0.105678990483284


### Sequence 2 info
SETDB1 is an H3K9 methyltransferase

H3K9 methylation indicates transcriptional repression  
KAP1 is a ubiquitous protein [likely involved in chromatin organization](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3143589/)

Cell types are largely cell lines/embronic